In [7]:
from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.devices import LocalSimulator
import numpy as np

# Garnet QPU 정의
device = AwsDevice("arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet")

# Bell state 회로 생성
bell_circuit = Circuit()
bell_circuit.h(0)
bell_circuit.cnot(control=0, target=1)

# 노이즈 파라미터 설정 (Garnet QPU 정보 기반)
t1 = 100e-6  # T1 시간 (예시 값, 실제 값은 더 길 수 있음)
t2 = 50e-6   # T2 시간 (예시 값, 실제 값은 더 길 수 있음)
single_qubit_gate_time = 20e-9  # 단일 큐비트 게이트 시간 (예시 값)
two_qubit_gate_time = 100e-9  # 두 큐비트 게이트 시간 (예시 값)
single_qubit_error = 1 - 0.9992  # 1 - 단일 큐비트 게이트 충실도
two_qubit_error = 1 - 0.9951  # 1 - 두 큐비트 게이트 충실도

# 노이즈 파라미터 계산
gamma1 = 1 / t1
gamma2 = 1 / t2 - gamma1 / 2

# 노이즈가 있는 회로 생성 (시뮬레이션용)
noisy_circuit = bell_circuit.copy()

# 단일 큐비트 게이트 노이즈 추가
for qubit in [0, 1]:
    noisy_circuit.amplitude_damping(qubit, gamma1 * single_qubit_gate_time)
    noisy_circuit.phase_damping(qubit, gamma2 * single_qubit_gate_time)
    noisy_circuit.depolarizing(qubit, single_qubit_error)

# CNOT 게이트에 대한 노이즈
noisy_circuit.amplitude_damping(0, gamma1 * two_qubit_gate_time)
noisy_circuit.amplitude_damping(1, gamma1 * two_qubit_gate_time)
noisy_circuit.phase_damping(0, gamma2 * two_qubit_gate_time)
noisy_circuit.phase_damping(1, gamma2 * two_qubit_gate_time)
noisy_circuit.two_qubit_depolarizing(0, 1, two_qubit_error)

# 밀도 행렬 시뮬레이터 정의
simulator = LocalSimulator("braket_dm")

# 결과를 저장할 딕셔너리
results = {}

# 이상적인 회로 실행
ideal_task = simulator.run(bell_circuit, shots=1000)
ideal_result = ideal_task.result()
results['ideal'] = ideal_result.measurement_counts

# 노이즈가 있는 회로 실행
noisy_task = simulator.run(noisy_circuit, shots=1000)
noisy_result = noisy_task.result()
results['noisy'] = noisy_result.measurement_counts

# 실제 Garnet QPU에서 실행
try:
    qpu_task = device.run(bell_circuit, shots=1000)
    qpu_result = qpu_task.result()
    results['qpu'] = qpu_result.measurement_counts
except Exception as e:
    results['qpu'] = f"실행 오류: {str(e)}"

# 결과 출력 및 분석
print("결과 딕셔너리:")
for key, value in results.items():
    print(f"{key}: {value}")

def analyze_results(results):
    for key, counts in results.items():
        if isinstance(counts, dict):
            total = sum(counts.values())
            print(f"\n{key} 결과 분석:")
            for state, count in counts.items():
                probability = count / total
                print(f"  |{state}>: {count} ({probability:.2%})")
        else:
            print(f"\n{key} 결과: {counts}")

analyze_results(results)



결과 딕셔너리:
ideal: Counter({'11': 524, '00': 476})
noisy: Counter({'11': 507, '00': 483, '10': 6, '01': 4})
qpu: Counter({'00': 507, '11': 467, '01': 17, '10': 9})

ideal 결과 분석:
  |11>: 524 (52.40%)
  |00>: 476 (47.60%)

noisy 결과 분석:
  |11>: 507 (50.70%)
  |00>: 483 (48.30%)
  |01>: 4 (0.40%)
  |10>: 6 (0.60%)

qpu 결과 분석:
  |00>: 507 (50.70%)
  |11>: 467 (46.70%)
  |01>: 17 (1.70%)
  |10>: 9 (0.90%)
